In [1]:
from google.colab import files
import zipfile
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from sklearn.svm import SVR

In [10]:
subject_info=files.upload()
proccessed_data=files.upload()

Saving subject-info.xlsx to subject-info (1).xlsx


Saving proccessed_data.zip to proccessed_data.zip


In [4]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')



Mounted at /content/drive


In [13]:


def mean_jump(file_path):
    # Load data from CSV file
    df = pd.read_csv(file_path)

    # Extract time and data from DataFrame
    time = df["Time [s]"]
    data = df["V_tidal [L]"]

    # Find local maxima and minima
    local_maxima = (np.diff(np.sign(np.diff(data))) < 0).nonzero()[0] + 1
    local_minima = (np.diff(np.sign(np.diff(data))) > 0).nonzero()[0] + 1

    # Ensure the number of maxima and minima are the same
    min_length = min(len(local_maxima), len(local_minima))
    local_maxima = local_maxima[:min_length]
    local_minima = local_minima[:min_length]

    # Calculate differences between consecutive maxima and minima
    differences = data.iloc[local_maxima].values - data.iloc[local_minima].values

    # Calculate average of differences
    average_difference = np.mean(differences)

    return average_difference

In [15]:
zip_filename = 'proccessed_data.zip'
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall('processed_data')


parent_directory_path = 'processed_data'

nested_directory_path = os.path.join(parent_directory_path, 'proccessed_data')

# Get a list of CSV files sorted by their names
csv_files = [file_name for file_name in os.listdir(nested_directory_path)]
csv_files.sort()

# Iterate through each CSV file, calculate the average, and store the results
averages = []
for file_name in csv_files:
    file_path = os.path.join(nested_directory_path, file_name)
    average_v_tidal = mean_jump(file_path)
    averages.append({'File': file_name, 'Average_V_tidal': average_v_tidal})

average_values = [entry['Average_V_tidal'] for entry in averages]
# Create a DataFrame from the averages list
data=pd.read_excel("subject-info (1).xlsx")

tidal_volume_series = pd.Series(average_values, name='tidal_volume')

# Assign the Series to the 'tidal_volume' column
data['tidal_volume'] = tidal_volume_series

data.to_excel("final_data_set.xlsx", index=False)

files.download("final_data_set.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
data=files.upload()

Saving final_data_set.xlsx to final_data_set (2).xlsx


In [45]:

data=pd.read_excel("final_data_set (2).xlsx")
data['Sex (M/F)']= data['Sex (M/F)'].replace('M',-1)
data['Sex (M/F)']= data['Sex (M/F)'].replace('F',1)
data['Asthma (Y/N)'] = data['Asthma (Y/N)'].replace('Y',1)
data['Asthma (Y/N)'] = data['Asthma (Y/N)'].replace('N',0)
data['History of Smoking (Y/N)'] = data['History of Smoking (Y/N)'].replace('N',0)
data['History of Smoking (Y/N)'] = data['History of Smoking (Y/N)'].replace('Y',1)
data['History of Vaping (Y/N)'] = data['History of Vaping (Y/N)'].replace('N',0)
data['History of Vaping (Y/N)'] = data['History of Vaping (Y/N)'].replace('Y',1)
data.replace('M ', -1, inplace=True)

# Feature selection/engineering
X = data[['Age [years]', 'Weight [kg]', 'Height [cm]', 'Chest Depth [mm]', 'Chest Width [mm]',
          'Sex (M/F)', 'Asthma (Y/N)', 'History of Smoking (Y/N)', 'History of Vaping (Y/N)']]
y = data['tidal_volume']



In [69]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=1)

# Model training
model = LinearRegression()
model.fit(X_train, y_train)

# Model evaluation
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)



Mean Absolute Error: 0.2675275531904294


In [65]:
# Define the number of times to augment the dataset
augmentation_factor = 1000

# Initialize an empty DataFrame to store augmented data
augmented_data = pd.DataFrame(columns=data.columns)


# Apply augmentation
for _ in range(augmentation_factor):
    # Apply augmentation techniques here
    # Example: Add random noise to numerical features
    augmented_sample = data.copy()
    for col in data.columns:
        if col == 'Age [years]':  # Check if the column is 'Age'
            # Convert 'Age' column to integer
            augmented_sample['Age [years]'] = augmented_sample['Age [years]'].astype(int)
        elif np.issubdtype(data[col].dtype, np.number):
            # Add random noise to numerical features
            augmented_sample[col] += np.random.normal(0, 0.1, size=len(data))

    # Append augmented sample to augmented data
    augmented_data = pd.concat([augmented_data, augmented_sample], ignore_index=True)

# Print the size of the final dataset
print("Original dataset size:", len(data))
print("Augmented dataset size:", len(augmented_data))




Original dataset size: 80
Augmented dataset size: 80000


In [66]:
print(augmented_data)

       Subject Number  Sex (M/F)  Height [cm]  Weight [kg] Age [years]  \
0            0.888867   0.955215   165.067377    59.239051          22   
1            2.094492   1.080644   172.101153    95.889907          23   
2            2.778951   1.155467   169.052907    66.107688          24   
3            3.909706  -0.975822   186.077960    85.575277          22   
4            4.758498  -1.017075   170.982854    61.000931          30   
...               ...        ...          ...          ...         ...   
79995       75.980778   1.055195   166.812177    56.102213          24   
79996       77.037578   1.064124   151.021806    41.776726          28   
79997       77.729057  -1.020084   180.816900    86.089210          23   
79998       79.084172  -1.147101   186.989899    85.015198          21   
79999       79.871799   1.132732   165.061048    54.005746          23   

       Asthma (Y/N)         Medication Dosage Frequency  \
0          0.224569                NaN              

We did augmentation here in order to increase our dataset thereby increasing our accuracy

In [74]:
# Feature selection/engineering
augmented_data["Age [years]"] = augmented_data['Age [years]'].astype(int)
X_augmented = augmented_data[['Age [years]', 'Weight [kg]', 'Height [cm]', 'Chest Depth [mm]', 'Chest Width [mm]',
          'Sex (M/F)', 'Asthma (Y/N)', 'History of Smoking (Y/N)', 'History of Vaping (Y/N)']]
y_augmented= augmented_data['tidal_volume']


In [70]:
X_train, X_test, y_train, y_test = train_test_split(X_augmented, y_augmented, test_size=0.2, random_state=1)

# Define a list of values for n_estimators to experiment with
n_estimators_values = [50, 100, 150, 200, 250,350,450,550,650,750,850,950,1000]

# Iterate over each value of n_estimators
for n_estimators in n_estimators_values:
    # Instantiate the Random Forest Regressor model with the current value of n_estimators
    model = RandomForestRegressor(n_estimators=n_estimators, random_state=42)

    # Train the model
    model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Calculate Mean Absolute Error (MAE)
    mae = mean_absolute_error(y_test, y_pred)

    # Print the MAE for the current value of n_estimators
    print(f"n_estimators = {n_estimators}, Mean Absolute Error: {mae}")

n_estimators = 50, Mean Absolute Error: 0.08305533410717278
n_estimators = 100, Mean Absolute Error: 0.08266682918370115
n_estimators = 150, Mean Absolute Error: 0.0824750813811108
n_estimators = 200, Mean Absolute Error: 0.08236409762442887
n_estimators = 250, Mean Absolute Error: 0.08233542614027277
n_estimators = 350, Mean Absolute Error: 0.08228541755895728
n_estimators = 450, Mean Absolute Error: 0.08223107675110898
n_estimators = 550, Mean Absolute Error: 0.08223550402875503
n_estimators = 650, Mean Absolute Error: 0.08221470993210422
n_estimators = 750, Mean Absolute Error: 0.0821863426046198
n_estimators = 850, Mean Absolute Error: 0.08217939117429128
n_estimators = 950, Mean Absolute Error: 0.08216155016348622
n_estimators = 1000, Mean Absolute Error: 0.08215998756137491


In [76]:
model = SVR(kernel='rbf')  # You can choose different kernels like 'linear', 'poly', 'rbf', etc.

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.24995713806142078
